In [1]:
import sys
sys.path.append("../../")
from src.utils import *
from shapely import wkt
from src.city_population import *
import censusdis.data as ced
alber_eq_us = "EPSG:5070"

In [2]:
osm_cities = gpd.read_file("../../data/osm_data_updated.csv") ## not pushed to github due to its size
osm_cities.geometry = osm_cities.geometry.apply(wkt.loads)
osm_cities = gpd.GeoDataFrame(osm_cities,geometry='geometry')
osm_cities = osm_cities.set_crs(epsg=4326)
osm_cities['country'] = [x.split(' ')[0] for x in osm_cities.locode]


In [3]:
city_gdf = osm_cities[osm_cities.country=='US']
city_gdf.head(3)

,geometry,place_id,osm_type,osm_id,lat,lon,class,type,place_rank,importance,addresstype,name,display_name,locode,bbox_west,bbox_south,bbox_east,bbox_north,country
0,"POLYGON ((-91.86484 33.93014, -91.86043 33.930...",297390788,relation,111100,33.9428794,-91.8434668,boundary,administrative,16,0.3897854173444353,city,Star City,"Star City, Lincoln County, Arkansas, 71667, Un...",US 5ST,-91.8648435,33.9191321,-91.8036945,33.9639797,US
44,"POLYGON ((-85.08881 43.81361, -85.08728 43.726...",299235226,relation,10055350,43.770084749999995,-85.02835864044233,boundary,administrative,14,0.4412922449371916,municipality,Coldwater Township,"Coldwater Township, Isabella County, Michigan,...",US CWA,-85.088808,43.726524,-84.968352,43.8137993,US
45,"POLYGON ((-85.56252 43.5675, -85.5625 43.56388...",298586541,relation,10058790,43.612498,-85.50289241285434,boundary,administrative,14,0.440956111033812,municipality,Mecosta Township,"Mecosta Township, Mecosta County, Michigan, Un...",US UMQ,-85.5625225,43.5548436,-85.4433126,43.6421546,US


In [4]:
city_pops_full = get_city_interpolation(city_gdf)

/home/kaushik/.local/share/virtualenvs/oef-hackday-q2-24-OS3fY_pt/lib/python3.9/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/kaushik/.local/share/virtualenvs/oef-hackday-q2-24-OS3fY_pt/lib/python3.9/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


Processing state: 00
Invalid state code: 00
Processing state: 01
Finished processing state: 01
Processing state: 02
Finished processing state: 02
Processing state: 03
Invalid state code: 03
Processing state: 04
Finished processing state: 04
Processing state: 05
Finished processing state: 05
Processing state: 06
Finished processing state: 06
Processing state: 07
Invalid state code: 07
Processing state: 08
Finished processing state: 08
Processing state: 09
Finished processing state: 09
Processing state: 10
Finished processing state: 10
Processing state: 11
Finished processing state: 11
Processing state: 12
Finished processing state: 12
Processing state: 13
Finished processing state: 13
Processing state: 14
Invalid state code: 14
Processing state: 15
Finished processing state: 15
Processing state: 16
Finished processing state: 16
Processing state: 17
Finished processing state: 17
Processing state: 18
Finished processing state: 18
Processing state: 19
Finished processing state: 19
Processi

In [5]:
city_pops_full = city_pops_full.dissolve(by = "place_id").reset_index()

In [6]:
city_pops_full.columns

Index(['place_id', 'geometry', 'osm_type', 'osm_id', 'lat', 'lon', 'class',
       'type', 'place_rank', 'importance', 'addresstype', 'name',
       'display_name', 'locode', 'bbox_west', 'bbox_south', 'bbox_east',
       'bbox_north', 'country', 'state', 'fip', 'interpolated_pop',
       'interpolated_households', 'interpolated_households_no_vehicle',
       'interpolated_workers'],
      dtype='object')

In [7]:
city_pops = city_pops_full[['place_id', 'addresstype', 'type', 'name', 'state', 'fip', 'interpolated_pop', 'interpolated_households', 'interpolated_households_no_vehicle', 'geometry']]
city_pops["interpolated_households_with_vehicle"] =  city_pops["interpolated_households"] - city_pops["interpolated_households_no_vehicle"]
city_pops.to_file('../../data/city_pops_full.geojson')


/home/kaushik/.local/share/virtualenvs/oef-hackday-q2-24-OS3fY_pt/lib/python3.9/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
